In [1]:
import torch
from torch.utils import data
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from datetime import datetime
from Nets import nets, PreActResNets, resnet_v2

TMA_array_path = '/data/Breast/TMA/bliss_data/17-002/Ki67[SP6]-Thermo'
onlyfiles = [f for f in listdir(TMA_array_path) if isfile(join(TMA_array_path, f)) and f.endswith('jpg')]

/home/dahen/miniconda3/envs/conda_master/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
inps = []
img_size = 256
for im_name in tqdm(onlyfiles):
    im = cv2.imread(join(TMA_array_path, im_name))
    (height,width)=im.shape[:2]
    im = im[int(height/2.0-img_size/2.0):int(height/2.0+img_size/2.0), 
            int(width/2.0-img_size/2.0):int(width/2.0+img_size/2.0),
            :]
    inps.append(np.transpose(im, axes=(2, 0, 1)))
inps = torch.FloatTensor(inps)

100%|███████████████████████████████████████████████████████████| 1188/1188 [00:23<00:00, 50.90it/s]
/tmp/ipykernel_126070/425953418.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484803030/work/torch/csrc/utils/tensor_new.cpp:201.)
  inps = torch.FloatTensor(inps)


In [3]:
inps = inps.repeat(10, 1, 1, 1)
print(inps.shape)
tgts = torch.ones(len(inps)*2, dtype=torch.float32).view(len(inps), 2)
dataset = data.TensorDataset(inps, tgts)

torch.Size([11880, 3, 256, 256])


In [4]:
device = torch.device('cuda') 
model = PreActResNets.PreActResNet50_Ron().to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loader = data.DataLoader(dataset, batch_size=129, pin_memory=True, num_workers=8)

start_time = datetime.now()

for batch_ndx, sample in enumerate(loader):
    # Every data instance is an input + label pair
    inputs, labels = sample
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # Make predictions for this batch
    outputs = model(inputs)[0][:, 0:2]

    # Compute the loss and its gradients
    loss = loss_fn(outputs, labels)
    loss.backward()

#         t = torch.cuda.get_device_properties(0).total_memory
#         r = torch.cuda.memory_reserved(0)
#         a = torch.cuda.memory_allocated(0)

#         print(f'total mem: {t}\n avail mem: {a}\n')

    # Adjust learning weights
    optimizer.step()
    
delta_time = datetime.now() - start_time
print(f"{len(loader)} batches took {delta_time} time, {delta_time/len(loader)} per batch")

PreActResNets.PreActResNet50_Ron()
93 batches took 0:01:54.006530 time, 0:00:01.225877 per batch
